# Исследование уровня потребительской лояльности

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Компания провела опрос пользователей.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).

Данные хранятся в базе.

**Наша задача:** 
- определить как распределены участники опроса по возрасту и полу 
- найти каких пользователей больше: новых или старых? 
- выяснить пользователи из каких городов активнее участвовали в опросе?
- какие группы пользователей наиболее лояльны к сервису? Какие менее?
- найти какой общий NPS среди всех опрошенных?
- описать клиентов, которые относятся к группе cторонников (англ. promoters)?

## Получение данных

In [8]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [14]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

Для начала посмотрим какие данные содержит база данных, выведем все данные.

In [18]:
query = """
select *
from User as u
left join location as loc ON u.location_id = loc.location_id
left join age_segment as ages ON u.age_gr_id = ages.age_gr_id
left join traffic_segment as tr on u.tr_gr_id = tr.tr_gr_id
left join lifetime_segment as lt on u.lt_gr_id = lt.lt_gr_id


"""

In [20]:
df = pd.read_sql(query, engine)
df.head(15)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,...,bucket_max,title,tr_gr_id,bucket_min,bucket_max,title,lt_gr_id,bucket_min,bucket_max,title
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,...,54.0,05 45-54,5,1.0,5.0,04 1-5,8,36.0,NaN,08 36+
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,...,54.0,05 45-54,5,1.0,5.0,04 1-5,8,36.0,NaN,08 36+
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,...,64.0,06 55-64,9,20.0,25.0,08 20-25,6,13.0,24.0,06 13-24
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,...,44.0,04 35-44,4,0.1,1.0,03 0.1-1,8,36.0,NaN,08 36+
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,...,24.0,02 16-24,6,5.0,10.0,05 5-10,8,36.0,NaN,08 36+
5,A005O0,5501,42.0,1.0,ANDROID,SMARTPHONE,34,4,6,8,...,44.0,04 35-44,6,5.0,10.0,05 5-10,8,36.0,NaN,08 36+
6,A0061R,1236,45.0,0.0,ANDROID,SMARTPHONE,55,5,7,8,...,54.0,05 45-54,7,10.0,15.0,06 10-15,8,36.0,NaN,08 36+
7,A009KS,313,35.0,0.0,ANDROID,SMARTPHONE,28,4,14,5,...,44.0,04 35-44,14,45.0,50.0,13 45-50,5,7.0,12.0,05 7-12
8,A00AES,3238,36.0,1.0,ANDROID,SMARTPHONE,41,4,5,8,...,44.0,04 35-44,5,1.0,5.0,04 1-5,8,36.0,NaN,08 36+
9,A00F70,4479,54.0,1.0,ANDROID,SMARTPHONE,9,5,8,8,...,54.0,05 45-54,8,15.0,20.0,07 15-20,8,36.0,NaN,08 36+


Посмотрим, какой объем данных содержит база данных.

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 26 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         502493 non-null  object 
 1   lt_day          502493 non-null  int64  
 2   age             501939 non-null  float64
 3   gender_segment  501192 non-null  float64
 4   os_name         502493 non-null  object 
 5   cpe_type_name   502493 non-null  object 
 6   location_id     502493 non-null  int64  
 7   age_gr_id       502493 non-null  int64  
 8   tr_gr_id        502493 non-null  int64  
 9   lt_gr_id        502493 non-null  int64  
 10  nps_score       502493 non-null  int64  
 11  location_id     502493 non-null  int64  
 12  city            502493 non-null  object 
 13  country         502493 non-null  object 
 14  age_gr_id       502493 non-null  int64  
 15  bucket_min      501254 non-null  float64
 16  bucket_max      488703 non-null  float64
 17  title     

В базе содержится информация о 502493 участниках опроса. Проверим есть пропуски в данных.

In [28]:
df.isna().sum()

user_id                0
lt_day                 0
age                  554
gender_segment      1301
os_name                0
cpe_type_name          0
location_id            0
age_gr_id              0
tr_gr_id               0
lt_gr_id               0
nps_score              0
location_id            0
city                   0
country                0
age_gr_id              0
bucket_min          1239
bucket_max         13790
title                  0
tr_gr_id               0
bucket_min             0
bucket_max         11105
title                  0
lt_gr_id               0
bucket_min             0
bucket_max        274299
title                  0
dtype: int64

Видим, что есть пропуски в возрасте и указании пола пользователей, а также пропуски в границах сегментов возраста, трафика и времени использования сервиса. 

Проверим какие значения содержатся в столбце "пол пользователя". 

In [31]:
df['gender_segment'].unique()

array([ 1.,  0., nan])

Видим, что в столбце есть три значения 1-женщины, 0 - мужчины и пропуски, которые при выгрузке мы заменим на значение "другое".

Проверим есть ли отрицательные значения в столбце `lt_day`

In [34]:
df[df['lt_day']<0]

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,...,bucket_max,title,tr_gr_id,bucket_min,bucket_max,title,lt_gr_id,bucket_min,bucket_max,title
7666,AEAC2R,-8,NaN,NaN,ANDROID,SMARTPHONE,50,8,8,8,...,NaN,08 n/a,8,15.0,20.0,07 15-20,8,36.0,NaN,08 36+
53997,CS0HF8,-2,NaN,NaN,ANDROID,SMARTPHONE,15,8,9,8,...,NaN,08 n/a,9,20.0,25.0,08 20-25,8,36.0,NaN,08 36+
71577,DORPT2,-4,NaN,NaN,ANDROID,SMARTPHONE,28,8,9,8,...,NaN,08 n/a,9,20.0,25.0,08 20-25,8,36.0,NaN,08 36+
100152,F5O3CG,-21,NaN,NaN,ANDROID,SMARTPHONE,60,8,12,8,...,NaN,08 n/a,12,35.0,40.0,11 35-40,8,36.0,NaN,08 36+
228087,LSE939,-13,NaN,NaN,ANDROID,SMARTPHONE,22,8,8,8,...,NaN,08 n/a,8,15.0,20.0,07 15-20,8,36.0,NaN,08 36+
325212,QTT7IR,-13,NaN,NaN,IOS,SMARTPHONE,28,8,7,8,...,NaN,08 n/a,7,10.0,15.0,06 10-15,8,36.0,NaN,08 36+
347784,RZUS6H,-6,NaN,NaN,ANDROID,SMARTPHONE,23,8,11,8,...,NaN,08 n/a,11,30.0,35.0,10 30-35,8,36.0,NaN,08 36+
385897,TYWQW4,-12,NaN,NaN,IOS,SMARTPHONE,33,8,5,8,...,NaN,08 n/a,5,1.0,5.0,04 1-5,8,36.0,NaN,08 36+
401226,URKAFI,-11,NaN,NaN,ANDROID,SMARTPHONE,30,8,7,8,...,NaN,08 n/a,7,10.0,15.0,06 10-15,8,36.0,NaN,08 36+
407160,V2GNK2,-13,NaN,NaN,IOS,SMARTPHONE,38,8,8,8,...,NaN,08 n/a,8,15.0,20.0,07 15-20,8,36.0,NaN,08 36+


Таких строк всего 13, при выгрузке мы их брать не будем.

Проверим столбец `age`.

In [37]:
df['age'].unique()

array([45., 53., 57., 44., 24., 42., 35., 36., 54., 39., 21., 27., 60.,
       34., 47., 37., 43., 33., 31., 25., 51., 28., 41., 40., 46., 48.,
       32., 30., 52., 59., 26., 50., 62., 29., 55., 22., 38., 56., 23.,
       49., 66., 74., 75., 17., 65., 64., 69., 58., 20., 19., 80., 70.,
       81., 63., 67., 68., 72., 15., 79., 18., 73., nan, 14., 71., 61.,
       16., 77., 13., 76., 10., 78., 12., 82., 11., 83., 89., 84., 85.,
       87., 86.])

В столбце `age` есть пропуски в значениях, их 554 строки, что составляет около 0.1% всех данных, поэтому при выгрузке пропустим эти значения.

Проверим значения в столбце `city`.

In [40]:
df['city'].sort_values().unique()

array(['Архангельск', 'Астрахань', 'Балашиха', 'Барнаул', 'Белгород',
       'Брянск', 'Владивосток', 'Владимир', 'Волгоград', 'Волжский',
       'Воронеж', 'Грозный', 'Екатеринбург', 'Иваново', 'Ижевск',
       'Иркутск', 'Казань', 'Калининград', 'Калуга', 'Кемерово', 'Киров',
       'Краснодар', 'Красноярск', 'Курск', 'Липецк', 'Магнитогорск',
       'Махачкала', 'Москва', 'НабережныеЧелны', 'НижнийНовгород',
       'НижнийТагил', 'Новокузнецк', 'Новосибирск', 'Омск', 'Оренбург',
       'Пенза', 'Пермь', 'РостовнаДону', 'Рязань', 'Самара',
       'СанктПетербург', 'Саранск', 'Саратов', 'Смоленск', 'Сочи',
       'Ставрополь', 'Сургут', 'Тверь', 'Тольятти', 'Томск', 'Тула',
       'Тюмень', 'УланУдэ', 'Ульяновск', 'Уфа', 'Хабаровск', 'Чебоксары',
       'Челябинск', 'Череповец', 'Чита', 'Якутск', 'Ярославль'],
      dtype=object)

Повторов в названиях городов нет.

Посмотрим какие значения находятся в столбце `nps_score`.

In [44]:
df['nps_score'].unique()

array([10,  6,  9,  2,  5,  8,  1,  4,  7,  3], dtype=int64)

Значения оценок находятся в диапазоне от 1 до 10.

Теперь мы можем выгружать нужные нам данные.

In [49]:
query = """
select user_id,lt_day,age,os_name,cpe_type_name,nps_score,
       loc.country,loc.city,
       substring(ages.title, 4) as age_segment,
    case 
        when gender_segment = 1 then 'female'
        when gender_segment = 0 then 'male'
    else 'others'
    end as gender_segment,
    case
        when lt_day <= 365 then 'new'
    else 'old'
    end as is_new,
    substring(tr.title, 4) as traffic_segment,
    substring(lt.title, 4) as lifetime_segment,
    case 
        when nps_score <= 6 then 'detractors'
        when nps_score >= 9 then 'promoters'
    else 'passives'
    end as nps_group
from User as u
left join location as loc ON u.location_id = loc.location_id
left join age_segment as ages ON u.age_gr_id = ages.age_gr_id
left join traffic_segment as tr on u.tr_gr_id = tr.tr_gr_id
left join lifetime_segment as lt on u.lt_gr_id = lt.lt_gr_id
Where lt_day >= 0 and age > 0

"""

In [51]:
df = pd.read_sql(query, engine)
df.head(15)

,user_id,lt_day,age,os_name,cpe_type_name,nps_score,country,city,age_segment,gender_segment,is_new,traffic_segment,lifetime_segment,nps_group
0,A001A2,2320,45.0,ANDROID,SMARTPHONE,10,Россия,Уфа,45-54,female,old,1-5,36+,promoters
1,A001WF,2344,53.0,ANDROID,SMARTPHONE,10,Россия,Киров,45-54,male,old,1-5,36+,promoters
2,A003Q7,467,57.0,ANDROID,SMARTPHONE,10,Россия,Москва,55-64,male,old,20-25,13-24,promoters
3,A004TB,4190,44.0,IOS,SMARTPHONE,10,Россия,РостовнаДону,35-44,female,old,0.1-1,36+,promoters
4,A004XT,1163,24.0,ANDROID,SMARTPHONE,10,Россия,Рязань,16-24,male,old,5-10,36+,promoters
5,A005O0,5501,42.0,ANDROID,SMARTPHONE,6,Россия,Омск,35-44,female,old,5-10,36+,detractors
6,A0061R,1236,45.0,ANDROID,SMARTPHONE,10,Россия,Уфа,45-54,male,old,10-15,36+,promoters
7,A009KS,313,35.0,ANDROID,SMARTPHONE,10,Россия,Москва,35-44,male,new,45-50,7-12,promoters
8,A00AES,3238,36.0,ANDROID,SMARTPHONE,10,Россия,СанктПетербург,35-44,female,old,1-5,36+,promoters
9,A00F70,4479,54.0,ANDROID,SMARTPHONE,9,Россия,Волгоград,45-54,female,old,15-20,36+,promoters


## Подготовка данных

Проверим данные на пропуски

In [54]:
df.isna().sum()

user_id             0
lt_day              0
age                 0
os_name             0
cpe_type_name       0
nps_score           0
country             0
city                0
age_segment         0
gender_segment      0
is_new              0
traffic_segment     0
lifetime_segment    0
nps_group           0
dtype: int64

Видим, что пропусков нет.

In [58]:
df['age'].unique()

array([45., 53., 57., 44., 24., 42., 35., 36., 54., 39., 21., 27., 60.,
       34., 47., 37., 43., 33., 31., 25., 51., 28., 41., 40., 46., 48.,
       32., 30., 52., 59., 26., 50., 62., 29., 55., 22., 38., 56., 23.,
       49., 66., 74., 75., 17., 65., 64., 69., 58., 20., 19., 80., 70.,
       81., 63., 67., 68., 72., 15., 79., 18., 73., 14., 71., 61., 16.,
       77., 13., 76., 10., 78., 12., 82., 11., 83., 89., 84., 85., 87.,
       86.])

In [60]:
df['gender_segment'].unique()

array(['female', 'male', 'others'], dtype=object)

## Создание дашбордов

Сохраним полученный датафрейм в файл для дальнейшей работы с ним в Tableau

In [63]:
df.to_csv('telecomm_csi_tableau_1.csv', index=False)

Дашборд "Участники опроса": https://public.tableau.com/views/2-_1_17131348708830/sheet5?:language=en-US&publish=yes&:sid=&:display_count=n&:origin=viz_share_link

Дашборд "Группы участников": https://public.tableau.com/views/2-_1_17131374453140/sheet10?:language=en-US&publish=yes&:sid=&:display_count=n&:origin=viz_share_link

Дашборд "Оценка NPS": https://public.tableau.com/views/2-NPS_1/NPS_7?:language=en-US&publish=yes&:sid=&:display_count=n&:origin=viz_share_link

Презентация: https://public.tableau.com/views/2-_1_17132225456620/Story1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link